In [1]:
## Import Libraries for processing the data
import pandas as pd
import numpy as np

In [2]:
## Read Excel file
with pd.ExcelFile("Test Data_wambui.xlsx") as file:
    # Read sheet with incomplete data
    Python_deal_data_incomplete_df = pd.read_excel(file,sheet_name="Python_deal data_incomplete")
    
    #Rean sheet with complete data
    SQL_deal_data_df = pd.read_excel(file,sheet_name="SQL_deal data")


In [3]:
## Display both dataframe to the console
print("==== Incomplete Dataframe ====")
print(Python_deal_data_incomplete_df)
print("==== Incomplete Dataframe ====")
print(SQL_deal_data_df)

==== Incomplete Dataframe ====
   deal_id      company_name deal_stage deal_value  deal_date
0     1001        Alpha Corp       Seed     500000 2024-07-01
1     1002          Beta Ltd   Series A       Seed 2024-02-01
2     1003        Beta L.T.D   Series B    3000000 2024-07-01
3     1004    Delta Partners   series-a     400000 2024-06-01
4     1005  Epsilon Ventures  Incubator       7000 2024-05-01
5     1006        Alpha Corp       Seed     500000 2024-07-01
==== Incomplete Dataframe ====
   deal_id      company_name deal_stage  deal_value  deal_date
0     1001        Alpha Corp       Seed      500000 2024-07-01
1     1002          Beta Ltd   Series A      200000 2024-02-01
2     1003          Beta Ltd   Series B     3000000 2024-01-01
3     1004    Delta Partners   Series A      400000 2024-06-01
4     1005  Epsilon Ventures  Incubator        7000 2024-05-01


In [4]:
## check data type in each column
Python_deal_data_incomplete_df.dtypes

deal_id                  int64
company_name            object
deal_stage              object
deal_value              object
deal_date       datetime64[ns]
dtype: object

In [5]:
## check data type in each column
SQL_deal_data_df.dtypes

deal_id                  int64
company_name            object
deal_stage              object
deal_value               int64
deal_date       datetime64[ns]
dtype: object

#### 1. Python script Identify variables that need cleaning

In [6]:
# define a function to return the column to be cleaned
def get_columns_to_clean(incorrect_df,correct_df):

    column_to_be_cleaned = []

    for col_incorrect_df,col_correct_df in zip(list(incorrect_df.columns),list(correct_df.columns)):
        # check if there is any column with missing values
        if incorrect_df[col_incorrect_df].isnull().any():
            # append the column within the list of columns te corrected
            column_to_be_cleaned.append(f'{col_incorrect_df}: has missing values')

        ## check the wrong data type by comparing data type between two columns
        if not incorrect_df[col_incorrect_df].dtype == correct_df[col_correct_df].dtype:
            column_to_be_cleaned.append(f'{col_incorrect_df}: has wrong data type, the correct type should:{correct_df[col_correct_df].dtype}')

        # Checck column with standardization issues
        # correct unique values from correct df
        if col_incorrect_df != "deal_id":
            correct_values = [str(val).strip() for val in  list(correct_df[col_correct_df])]
            incorrect_values = [str(val).strip() for val in  list(incorrect_df[col_incorrect_df].unique())]

            if not pd.Series(incorrect_values).isin(correct_values).all():
                column_to_be_cleaned.append(f'{col_incorrect_df}: has standardization issue')      
    return column_to_be_cleaned



In [7]:
## Invoke the functions to displaying columns to be cleaned, the script will print also custom message higligh the issue type
print(get_columns_to_clean(Python_deal_data_incomplete_df,SQL_deal_data_df))

['company_name: has standardization issue', 'deal_stage: has standardization issue', 'deal_value: has wrong data type, the correct type should:int64', 'deal_value: has standardization issue']


#### 2. Clean the deal_date variable

In [ ]:
## Define python to clean deal_date column
def clean_date(date):
    try:
        # Attempt to parse using the specified format
        parsed_date = pd.to_datetime(date, errors='coerce', format="%m/%Y")
    except:
        # Handle non-standard formats
        parsed_date = pd.to_datetime(date, errors='coerce')
    
    # Return the date as a formatted string if valid, otherwise None
    return parsed_date.strftime("%m/%Y") if pd.notna(parsed_date) else None


# Call the function for cleanind deal_date
Python_deal_data_incomplete_df['deal_date'] = Python_deal_data_incomplete_df['deal_date'].apply(clean_date)

In [9]:
## check the values in deal_date after cleaning
Python_deal_data_incomplete_df['deal_date']

0    07/2024
1    02/2024
2    07/2024
3    06/2024
4    05/2024
5    07/2024
Name: deal_date, dtype: object

#### 3. Remove duplicates

In [ ]:
## check duplicate based by combining variables which are 'company_name', 'deal_stage', 'deal_value', 'deal_date'
Python_deal_data_incomplete_df[Python_deal_data_incomplete_df.duplicated(subset=['company_name', 'deal_stage', 'deal_value', 'deal_date'],keep=False)]

,deal_id,company_name,deal_stage,deal_value,deal_date
0,1001,Alpha Corp,Seed,500000,07/2024
5,1006,Alpha Corp,Seed,500000,07/2024


In [ ]:
## It seems like we have two duplicate entries
# keep the first
Python_deal_data_incomplete_df = Python_deal_data_incomplete_df.drop_duplicates(subset=['company_name', 'deal_stage', 'deal_value', 'deal_date'],keep='first')

## Display the dataframe after removing duplicate
Python_deal_data_incomplete_df

,deal_id,company_name,deal_stage,deal_value,deal_date
0,1001,Alpha Corp,Seed,500000,07/2024
1,1002,Beta Ltd,Series A,Seed,02/2024
2,1003,Beta L.T.D,Series B,3000000,07/2024
3,1004,Delta Partners,series-a,400000,06/2024
4,1005,Epsilon Ventures,Incubator,7000,05/2024


#### 4. Create a new variable called “latest_deal_stage” to show the latest deal by date

In [ ]:
## Create a function to show the latest deal
def get_python_date_value(date):
    # convert date string into python date object
     # Attempt to parse using the specified format
    parsed_date = pd.to_datetime(date, errors='coerce', format="%m/%Y")
    return parsed_date

#get the latest date python object
latest_deal_date = max(list(Python_deal_data_incomplete_df['deal_date'].apply(get_python_date_value)))

## convert value back to a date string to standardize the value from use case
latest_deal_stage = latest_deal_date.strftime("%m/%Y")
latest_deal_stage


'07/2024'